In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from keras import backend as K
 
img_width, img_height = 128, 128

input_shape = (img_width, img_height, 3) ## we checked that the RGB channel comes last in the data format of these images

epochs = 10
batch_size = 8

### Creating the model

In [10]:
model = Sequential()
model.add(Conv2D(32, (2, 2), activation = 'relu', input_shape = input_shape)) ## convolution of kernel with image
model.add(MaxPooling2D(pool_size=(2, 2))) ## reduces dimensionality in order to reduce computational cost + minimize overfitting
 
model.add(Conv2D(32, (2, 2), activation = 'relu'))  ## convolution of kernel with image
model.add(MaxPooling2D(pool_size=(2, 2)))  ## reduces dimensionality in order to reduce computational cost + minimize overfitting
 
model.add(Conv2D(64, (2, 2), activation = 'relu'))  ## convolution of kernel with image
model.add(MaxPooling2D(pool_size=(2, 2)))  ## reduces dimensionality in order to reduce computational cost + minimize overfitting

model.add(Flatten()) ## transitions from convolutional to fully connected layers by converting the input from multi-dimensional --> 1D
model.add(Dense(64, activation = 'relu')) ## creates a fully connected layer with 64 output units (neurons)
model.add(Dropout(0.1)) ## this function randomly sets 10% of the input units to 0 at each training step; used to prevent overfitting
model.add(Dense(4, activation = 'sigmoid')) ## creates a fully connected layer with 4 output units (neurons) corresponding to the 4 output classes

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy']) ## try accuracy for now, can change later

### Building the model (training on train data and building it to maximize accuracy on validation data)

In [15]:
train_data = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\GitHub\BMEN-415\Data\_split datasets\image train_test_val split\train',
    image_size = (img_width, img_height),
    batch_size = batch_size,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented'],
    seed = 112)
 
validation_data = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\GitHub\BMEN-415\Data\_split datasets\image train_test_val split\val',
    image_size = (img_width, img_height),
    batch_size = batch_size,
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented'],
    seed = 112)
 
model.fit(
    train_data,
    steps_per_epoch = 4487 // batch_size,
    epochs=epochs,
    validation_data = validation_data,
    validation_steps = 642 // batch_size,
    class_weight = {0: 4487/627, 1: 4487/44, 2: 4487/2240, 3: 4487/1568}) ## these class weights are based on the train dataset (total train samples/train samples for class i)

Found 4487 files belonging to 4 classes.
Found 642 files belonging to 4 classes.
Epoch 1/10
280/280 [==============================] - 70s 251ms/step - loss: 2.0440 - accuracy: 0.5179 - val_loss: 0.7257 - val_accuracy: 0.4984
Epoch 2/10
280/280 [==============================] - 3s 9ms/step - loss: 3.3089 - accuracy: 0.7143 - val_loss: 0.5916 - val_accuracy: 0.2125


In [ ]:
model.save('CNN_model.h5')

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np
 
from keras.models import load_model
 
model = load_model('CNN_model.h5')
 
## let's test one of the mild demented images reserved in the training dataset
image = load_img(r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\_FINAL DATASETS\alzheimer mri preprocessed dataset\train_test_val split\test\Mild_Demented\mild_55.jpg', target_size = (128, 128))
img = np.array(image) / 255
img = img.reshape(1, 128, 128, 3)

prediction = model.predict(img)
print(prediction)
print("Predicted Class: ", prediction[0][0])

[[0.25720322]]
Predicted Class:  0.25720322
